In [1]:
import cv2
from numpy import *
from scipy import ndimage
from copy import deepcopy

In [22]:
scaling = 1
img1 = cv2.imread('im_0.jpg')
img2 = cv2.imread('im_1.jpg')
img1D = cv2.imread('depth_0.jpg',0)
img2D = cv2.imread('depth_1.jpg',0)

depth1 = cv2.resize(img1D,(0,0),fx = scaling, fy = scaling)
depth2 = cv2.resize(img2D,(0,0), fx = scaling, fy = scaling)
img1 = cv2.resize(img1,(0,0), fx = scaling, fy = scaling)
img2 = cv2.resize(img2,(0,0), fx = scaling, fy = scaling)

gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
print shape(gray1)




(360, 640)


In [23]:
def createdict(depth1):
    d = zeros([256])
    rows,cols = depth1.shape[0],depth1.shape[1]
    for i in range(rows):
        for j in range(cols): 
            d[depth1[i,j]]+=1
            
        
    no = (rows*cols)//5
    return d,no
    


        

    

In [24]:
def interpolate(img):
    rows,cols = img.shape[0],img.shape[1]
    dst = img.copy()
    for i in range(1,rows-1): 
        for j in range(1,cols-1): 
            if(img[i,j].tolist()==[0,0,0]):
                isolate = img[i-1:i+2,j-1:j+2].reshape(9,3)
                for m in range(9): 
                    if(isolate[m].tolist()!=[0,0,0]):
                        dst[i,j] = isolate[m]
   
    return dst.astype('uint8')
    

In [25]:
def quantize(img,l,depth1):
    
    l = [50,100,150,200]
    rows,cols = depth1.shape[0],depth1.shape[1]
    img_dis_1 = zeros([rows,cols,3],dtype = 'uint8')
    img_dis_2 = zeros([rows,cols,3],dtype = 'uint8')
    img_dis_3 = zeros([rows,cols,3],dtype = 'uint8')
    img_dis_4 = zeros([rows,cols,3],dtype = 'uint8')
    img_dis_5 = zeros([rows,cols,3],dtype = 'uint8')
    for i in range(rows):
        for j in range(cols): 
            if(depth1[i,j]<l[0]):
                img_dis_1[i,j] = img1[i,j]
            if(depth1[i,j]>l[0] and depth1[i,j]<l[1]):
                img_dis_2[i,j] = img1[i,j]
            if(depth1[i,j]>l[1] and depth1[i,j]<l[2]): 
                img_dis_3[i,j] = img1[i,j]
            if(depth1[i,j]>l[2] and depth1[i,j]<l[3]):
                img_dis_4[i,j] = img1[i,j]
            if(depth1[i,j]>l[3]): 
                img_dis_5[i,j] = img1[i,j]
                
                
        
    return img_dis_1,img_dis_2,img_dis_3,img_dis_4,img_dis_5
    

In [26]:
def createImage(final,img,val): 
    rows,cols = img.shape[0],img.shape[1]
    out = zeros([rows,cols,3],dtype = 'uint8')
    for i in range(rows):
        for j in range(cols): 
            if(final[i,j]==val):
                out[i,j] = img[i,j]
                
    return out

In [27]:
def calculatehomography(rand_pts,src,dst): 
   
    A = zeros([8,9])
    for i in range(4):
            pt = int(rand_pts[i])
            s = src[pt]
            d = dst[pt]
            A[2*i]   = [-s[0],-s[1],-s[2],0,0,0,d[0]*s[0],d[0]*s[1],d[0]*s[2]]
            A[2*i+1] = [0,0,0,-s[0],-s[1],-s[2],d[1]*s[0],d[1]*s[1],d[1]*s[2]]
            
    u, s, v = linalg.svd(A)

    #reshape the min singular value into a 3 by 3 matrix
    h = reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h
    

In [28]:
def obtainDistanceTransform(img): 
    rows = img.shape[0]
    cols = img.shape[1]
    ret = zeros([rows,cols,3])
    a =  ones([rows,cols])
    b = zeros([rows+2,cols+2])
    b[1:-1,1:-1] = a
    
    c = ndimage.distance_transform_edt(b)
    d = c[1:-1,1:-1]
    [ret[:,:,0],ret[:,:,1],ret[:,:,2]] = [d,d,d] 

    return ret

In [29]:
def StitchRGBD(img1,l,depth1,img2):
    
    img_dis_1,img_dis_2,img_dis_3,img_dis_4,img_dis_5 = quantize(img1,l,depth1)
    
    H1,H2,H3,H4,H5 = HomoD(img2,img1,depth1,3,l)

   

    dist1 = obtainDistanceTransform(img_dis_2)
    dist2 = obtainDistanceTransform(img_dis_2) 
    dist3 = obtainDistanceTransform(img_dis_3)
    dist4 = obtainDistanceTransform(img_dis_4)
    dist5 = obtainDistanceTransform(img_dis_5)


    src5,des5 = warp_image(img_dis_5,H5)
    src4,des4 = warp_image(img_dis_4,H4)
    src3,des3 = warp_image(img_dis_3,H3)
    src2,des2 = warp_image(img_dis_2,H2)
    src1,des1 = warp_image(img_dis_1,H2)



    mini_col = 0
    mini_row = 0
    max_row = 0
    max_col = 0

    for i in range(len(des1)):
        col = des1[i][0]
        row = des1[i][1]

        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row

    for i in range(len(des5)):
        col = des5[i][0]
        row = des5[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row

    for i in range(len(des4)):
        col = des4[i][0]
        row = des4[i][1]
        if(col<mini_col):
            mini_col = col
        if(col>max_col):
            max_col = col
        if(row>max_row):
            max_row = row
        if(row<mini_row):
            mini_row = row

    for i in range(len(des3)):
        col = des3[i][0]
        row = des3[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row

    for i in range(len(des2)):
        col = des2[i][0]
        row = des2[i][1]
        if(col<mini_col):

            mini_col = col

        if(col>max_col):

            max_col = col

        if(row>max_row):

            max_row = row

        if(row<mini_row):

            mini_row = row




    final_rows = int(max_row-mini_row)+1
    final_cols = int(max_col-mini_col)+1

    distance_1 = ones([final_rows,final_cols,3])
    distance_2 = ones([final_rows,final_cols,3])
    distance_3 = ones([final_rows,final_cols,3])
    distance_4 = ones([final_rows,final_cols,3])
    distance_5 = ones([final_rows,final_cols,3])

    final = zeros([final_rows,final_cols,3])

    I1 = zeros([final_rows,final_cols,3])
    I2 = zeros([final_rows,final_cols,3])
    I3 = zeros([final_rows,final_cols,3])
    I4 = zeros([final_rows,final_cols,3])
    I5 = zeros([final_rows,final_cols,3])

    for i in range(len(des1)):
        row = des1[i][1]
        col = des1[i][0]
        val1 = img_dis_1[int(src1[i][1]),int(src1[i][0])]
        val2 = dist1[int(src1[i][1]),int(src1[i][0])]
        distance_1[row-mini_row,col-mini_col] = val2
        I1[row-mini_row,col-mini_col] = val1
        final[row-mini_row,col-mini_col] = val1


    for i in range(len(des5)):
        row = des5[i][1]
        col = des5[i][0]
        val1 = img_dis_5[int(src5[i][1]),int(src5[i][0])]
        val2 = dist5[int(src5[i][1]),int(src5[i][0])]
        distance_5[row-mini_row,col-mini_col] = val2
        I5[row-mini_row,col-mini_col] = val1
        final[row-mini_row,col-mini_col] = val1

    for i in range(len(des4)):
        row = des4[i][1]
        col = des4[i][0]
        val1 = img_dis_4[int(src4[i][1]),int(src4[i][0])]
        val2 = dist4[int(src4[i][1]),int(src4[i][0])]
        distance_4[row-mini_row,col-mini_col] = val2
        I4[row-mini_row,col-mini_col] = val1
        final[row-mini_row,col-mini_col] = val1

    for i in range(len(des3)):
        row = des3[i][1]
        col = des3[i][0]
        val1 = img_dis_3[int(src3[i][1]),int(src3[i][0])]
        val2 = dist3[int(src3[i][1]),int(src3[i][0])]
        distance_3[row-mini_row,col-mini_col] = val2
        I3[row-mini_row,col-mini_col] = val1
        final[row-mini_row,col-mini_col] = val1

    for i in range(len(des2)):
        row = des2[i][1]
        col = des2[i][0]
        val1 = img_dis_2[int(src2[i][1]),int(src2[i][0])]
        val2 = dist2[int(src2[i][1]),int(src2[i][0])]
        I2[row-mini_row,col-mini_col] = val1
        final[row-mini_row,col-mini_col] = val1
        distance_2[row-mini_row,col-mini_col] = val2

    I1 = interpolate(I1.astype('uint8'))
    #print "interpolated 1" 
    I2 = interpolate(I2.astype('uint8'))
    #print "interpolated 2"
    I3 = interpolate(I3.astype('uint8'))
    #print "interpolated 3"
    I4 = interpolate(I4.astype('uint8'))
    #print "interpolated 4"
    I5 = interpolate(I5.astype('uint8'))
    #I_ret = interpolate(final.astype('uint8'))
    for i in range(I1.shape[0]): 
        for j in range(I1.shape[1]):
            if(I1[i,j].tolist()!=[0,0,0]):
                final[i,j] = I1[i,j]
            elif(I2[i,j].tolist()!=[0,0,0]):
                final[i,j] = I2[i,j]
            elif(I3[i,j].tolist()!=[0,0,0]):
                final[i,j] = I3[i,j]
            elif(I4[i,j].tolist()!=[0,0,0]):
                final[i,j] = I4[i,j]
            elif(I5[i,j].tolist()!=[0,0,0]):
                final[i,j] = I5[i,j]
    
    return final



In [30]:
def calculate_norm(array1,array2): 
    diff = subtract(array1,array2)
    return (diff[0]**2+diff[1]**2+diff[2]**2)**0.5

In [31]:
def warp_image(img2,homo):
    all_src = []
    #print shape(homo)
    for i in range(img2.shape[0]):
        for j in range(img2.shape[1]): 
            all_src.append([j,i,1])
    all_src = asarray(all_src)
    all_src_trans = all_src.T
    all_des_trans = normalize(dot(homo,all_src_trans))
    all_des = all_des_trans.T
    all_des = all_des.astype('int32')
    return all_src,all_des

In [32]:
def Warped(img1,img2):
    src_pts,des_pts = MatchKeypoints(img1,img2)
    
    H21 = estimatehomography(img1,img2,3)

    src2,des2 = warp_image(img2,H21)
    
    mini_col = 0
    mini_row = 0
    max_row = 0
    max_col = 0
    for i in range(len(des2)): 
        col = des2[i][0]
        row = des2[i][1]

        if(row<mini_row):
            mini_row = row

        if(col<mini_col):
            mini_col = col

        if(row>max_row):
            max_row = row

        if(col>max_col):
            max_col = col

   

    final_rows = max_row-mini_row+1
    final_cols = max_col-mini_col+1
    print final_rows,final_cols

    final_image = zeros([final_rows,final_cols,3])
    I1 = zeros([final_rows,final_cols,3])
    I2 = zeros([final_rows,final_cols,3])

    
    



    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            if(i-mini_row<final_rows and j-mini_col<final_cols):
                I1[i-mini_row,j-mini_col] = img1[i,j]
                



    for i in range(len(des2)):
        row = des2[i][1]
        col = des2[i][0]
        val1 = img2[int(src2[i][1]),int(src2[i][0])]
        I2[row-mini_row,col-mini_col] = val1
       


   
    I2 = interpolate(I2.astype('uint8'))
    
    
   
    
    
    return I2

In [33]:
def ConvertArray(src_pts,dst_pts): 
    src = []
    des = []
    for i in src_pts: 
        k = i.ravel().tolist()
        k.append(1)
        src.append(k)

    for i in dst_pts: 
        k = i.ravel().tolist()
        k.append(1)
        des.append(k)

    src = asarray(src)
    des = asarray(des)
    return src,des





In [34]:
def findgoodmatches(matches):
    good = []
    for m,n in matches: 
        if m.distance<0.75*n.distance:
            good.append(m)
    return good


In [35]:
def MatchKeypoints(img1,img2): 
    gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    bf = cv2.BFMatcher()
    kp1,des1 = sift.detectAndCompute(gray1,None)
    kp2,des2 = sift.detectAndCompute(gray2,None)
    matches = bf.knnMatch(des2,des1,k=2)
    
    
    good = findgoodmatches(matches)
    
    
    src_pts = float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = float32([kp1[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    
    return src_pts,dst_pts
    
    

In [36]:
def normalize(mat): 
    if(0 not in mat[-1]):
        return mat/mat[-1]
    else: 
        return zeros([3,len(mat[1])])

In [37]:
def checkpoint(depth1):
    d,no = createdict(depth1)
    l = []
    sum1 = 0
    for i in range(len(d)):
        sum1+=d[i]
        if(sum1>no):
            l.append(i)
            sum1 = 0

    return l



In [38]:
def interpolate_dist(img):
    rows,cols = img.shape[0],img.shape[1]
    dst = img.copy()
    for i in range(1,rows-1): 
        for j in range(1,cols-1): 
            if(img[i,j].tolist()==[1,1,1]):
                isolate = img[i-1:i+2,j-1:j+2].reshape(9,3)
                for m in range(9): 
                    if(isolate[m].tolist()!=[1,1,1]):
                        dst[i,j] = isolate[m]
   
    return dst

In [39]:
def ransac(src,dst,threshold):
    #First we choose any 4 random points and estimate the homography
    
    max_cnt = 0
    max_inlier = []
    number = len(src)
    threshold_inliers = int(0.90*len(src))
    best_homo = zeros([3,3])
    for ite in range(7000):
       
        rand_pts = random.randint(len(src),size = (4,)) 
        inliers = []
        final = []
        H = calculatehomography(rand_pts,src,dst)
        transposed_src = src.T
        transposed_des = normalize(dot(H,transposed_src))
        inliers = []
        expected_des = dst.T
        distance = calculate_norm(expected_des,transposed_des)
        for i in range(len(distance)): 
            if(distance[i]<threshold):
                inliers.append(i)
            if(len(inliers)>threshold_inliers): 
                max_inlier = inliers
                max_cnt = len(inliers)
                best_homo = H
                break
            elif(len(inliers)>max_cnt):
                max_cnt = len(inliers)
                max_inlier = inliers 
                best_homo = H
                
    
    return best_homo
        
                
    




    
        
                
                
        
        
    
        
    
        
        
    

In [40]:
def estimatehomography(img1,img2,threshold):
    
    src_pts, dst_pts = MatchKeypoints(img1,img2)
    src, dst = ConvertArray(src_pts,dst_pts)
    homo = ransac(src,dst,threshold)
    
    return homo
    ##Now apply RANSAC on the image. 
    
    
    

In [43]:
def HomoD(img1,img2,depth2,threshold,l):
    
    src_pts, dst_pts = MatchKeypoints(img1,img2)
    src,des = ConvertArray(src_pts, dst_pts)
    des1,src1 = [],[]
    des2,src2 = [],[]
    des3,src3 = [],[]
    des4,src4 = [],[]
    des5,src5 = [],[]
    for i in range(len(src_pts)):
        row = src[i][1]
        col = src[i][0]
        rowd = des[i][1]
        cold = des[i][0]
        val = depth2[int(row),int(col)]
        if(val<l[0]):
            des1.append([cold,rowd,1])
            src1.append([col,row,1])
        if(val>l[0] and val<l[1]): 
            des2.append([cold,rowd,1])
            src2.append([col,row,1])
        if(val>l[1] and val<l[2]):
            des3.append([cold,rowd,1])
            src3.append([col,row,1])
        if(val>l[2] and val<l[3]):
            des4.append([cold,rowd,1])
            src4.append([col,row,1])
        if(val>l[3]):
            des5.append([cold,rowd,1])
            src5.append([col,row,1]) 
            
    src1,des1 = asarray(src1),asarray(des1)
    src2,des2 = asarray(src2),asarray(des2)
    src3,des3 = asarray(src3),asarray(des3)
    src4,des4 = asarray(src4),asarray(des4)
    src5,des5 = asarray(src5),asarray(des5)
    homo1 = ransac(src1,des1,threshold)
    homo2 = ransac(src2,des2,threshold)
    homo3 = ransac(src3,des3,threshold)
    homo4 = ransac(src4,des4,threshold)
    homo5 = ransac(src5,des5,threshold)
    
    return homo1,homo2,homo3,homo4,homo5


In [44]:
#Q2(d) Obtaining the homography at each of the depth levels. 
h1,h2,h3,h4,h5 = HomoD(img2,img1,depth1,3,l)

print
print "level 1"
print h1
print 
print "level 2"
print h2
print 
print "level 3"
print h3
print
print "level 4"
print h4
print
print "level 5"
print h5


level 1
[[ 8.25464749e-01  1.21598073e-01  5.79487527e+01]
 [-8.48399823e-02  9.81829460e-01 -3.19637862e+01]
 [-3.53819850e-04  2.17409998e-04  1.00000000e+00]]

level 2
[[ 8.91629450e-01  1.50360623e-01  5.12900359e+01]
 [-7.50556062e-02  1.04154236e+00 -3.98311845e+01]
 [-2.85796265e-04  3.31709137e-04  1.00000000e+00]]

level 3
[[ 8.39753266e-01  1.11100450e-01  6.64132336e+01]
 [-8.77003855e-02  9.77952701e-01 -2.65879566e+01]
 [-3.20584180e-04  2.34741471e-04  1.00000000e+00]]

level 4
[[ 7.83137098e-01 -7.16695274e-03  9.82223408e+01]
 [-5.56974634e-02  8.22393010e-01 -9.24076691e+00]
 [-2.29729281e-04 -1.03305255e-04  1.00000000e+00]]

level 5
[[ 3.61896496e-02 -9.16703765e-01  3.39589753e+02]
 [-4.66330779e-02 -6.40616390e-01  2.61426510e+02]
 [-1.62601068e-04 -2.49034071e-03  1.00000000e+00]]


In [45]:
#Q2(e) Obtaining the final warped image

l = checkpoint(depth1)
final = StitchRGBD(img1,l,depth1,img2)
cv2.imshow('final',final.astype('uint8'))
cv2.waitKey(0)

-1

In [47]:
#Q2(f) Comparing results with original. 
#Warping the results with the original and obtaining

final_image = Warped(img2,img1)
cv2.imshow('final_image',final_image)
cv2.waitKey(0)

406 756


-1